In [1]:
#
# Steady-state density matrix of a two-level atom in a high-Q
# cavity for various driving frequencies calculated using
# iterative 'steady' solver.
#
# Adapted from 'probss' example in the qotoolbox by Sze M. Tan.
#
from qutip import *
from pylab import *


def probss(E, kappa, gamma, g, wc, wa, wl, N):
    # construct composite operators
    ida = qeye(N)
    idatom = qeye(2)
    a = tensor(destroy(N), idatom)
    sm = tensor(ida, sigmam())
    sp = tensor(ida, sigmap())
    
    # Hamiltonian
    H = (wl - wl) * sp * sm + (wc - wl) * a.dag() * a + \
        g * (a.dag() * sm + sp * a) + E * (a.dag() + a)#+  E*(sm+sm.dag())#   

    # Collapse operators
    C1 = sqrt(2 * kappa) * a
    C2 = sqrt(2 * gamma) * sm
    C1dC1 = C1.dag() * C1
    C2dC2 = C2.dag() * C2

    # find steady state
    rhoss = steadystate(H, [C1, C2],use_precond=True, 
                method='iterative-bicgstab', use_rcm = False, tol=1e-15)

    # calculate expectation values
    count1 = expect(C1dC1, rhoss)
    count2 = expect(C2dC2, rhoss)
    count3 = expect(sm,rhoss)
    infield = expect(a, rhoss)
    spa = expect(a*sp,rhoss)
    sma = expect(a.dag()*sm,rhoss)
    return count1, count2, count3, sma, spa, infield


# setup the calculation
#-----------------------
# must be done before parfunc unless we
# want to pass all variables as one using
# zip function (see documentation for an example)
kappa = 0.1
gamma = 0.1
g = 3.0
wc = 0.0
wa = 0.0
N =20
E = 0.1280
nloop = 4000
wlist = linspace(-4.5,4.5,nloop)

# define single-variable function for use in parfor
# cannot be defined inside run() since it needs to
# be passed into seperate threads.
def parfunc(wl):  # function of wl only
    count1, count2, count3, sma, spa, infield = probss(E, kappa, gamma, g, wc, wa, wl, N)
    return count1, count2, count3, sma, spa, infield


def run():
    # run parallel for-loop over wlist
    count1, count2, count3, sma, spa, infield = parfor(parfunc, wlist)
    
    output_data=vstack((wlist,real(count1), imag(count1),real(count2),imag(count2),real(count3),imag(count3),real(sma),imag(sma),real(infield),imag(infield)))
   # output_data=vstack((wlist,real(count3),imag(count3),real(infield),imag(infield),real(sma), imag(sma),real(spa), imag(spa)))
    file_data_store('/home/photon/Dropbox/data/two_level/borderline_g3_k_gam_0_1_eta1_50_vertical_scan.csv',output_data.T,numtype="real",numformat="decimal",sep=",")
    
    # plot cavity emission and qubit spontaneous emssion
    fig = figure(1)
    ax = fig.add_subplot(111)
    ax.plot(wlist, real(sma), lw=2)
    ax.plot(wlist, imag(sma), wlist, imag(sma), wlist, imag(spa), wlist, imag(spa),lw=2)
    xlabel('Drive Frequency Detuning')
    ylabel('Count rates')
    show()

    # plot phase shift of cavity light
    #fig2 = figure(2)
    #ax2 = fig2.add_subplot(111)
    #ax2.plot(wlist, imag(sma), wlist, imag(count3), wlist, real(infield), lw=2)
    #xlabel('Drive Frequency Detuning')
    #ylabel('Intracavity phase shift')
    #show()


if __name__ == "__main__":
    run()

<Figure size 640x480 with 1 Axes>